<a href="https://colab.research.google.com/github/ANGB022210151/AquacultureProject/blob/main/feature_engineering_Step2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# 1. Reload the dataset into a DataFrame named `df`
df = pd.read_csv('/content/fault_detected_sensor_data.csv')

# 2. Pre-process the 'time' column
df['time'] = df['time'].str.replace(r'\s+\(.*?\)', '', regex=True)
df['time'] = pd.to_datetime(df['time'], format="%a %b %d %Y %H:%M:%S GMT%z")

# 3. Sort the DataFrame `df` by the 'time' column and reset the index.
df = df.sort_values(by='time').reset_index(drop=True)

# 4. Calculate the time difference between consecutive readings in seconds
df['time_diff_seconds'] = df['time'].diff().dt.total_seconds()



# 5. Create 'temperature_cleaned' column
df['temperature_cleaned'] = df['Temperature (°C)'].copy()
df['temperature_cleaned'] = df['temperature_cleaned'].replace(-999, np.nan)
df['temperature_cleaned'] = df['temperature_cleaned'].ffill()

# 6. Create 'tds_cleaned' column
df['tds_cleaned'] = df['Time Dissolve Solid (ppm)'].copy()
df['tds_cleaned'] = df['tds_cleaned'].replace(0, np.nan)
df['tds_cleaned'] = df['tds_cleaned'].ffill()

# Create pH_cleaned column
df['pH_cleaned'] = df['pH'].copy()
# Replace invalid pH values with NaN
df.loc[(df['pH_cleaned'] <= 0) | (df['pH_cleaned'] > 14), 'pH_cleaned'] = np.nan
# Forward-fill missing values
df['pH_cleaned'] = df['pH_cleaned'].ffill()

# Create turbidity_cleaned column
df['turbidity_cleaned'] = df['Turbidity (NTU)'].copy()
df['turbidity_cleaned'] = df['turbidity_cleaned'].replace(999, np.nan)
df['turbidity_cleaned'] = df['turbidity_cleaned'].ffill()

# 7. Recalculate the 'Rate of Change (ΔT/Δt)' using 'temperature_cleaned'
df['Rate of Change (ΔT/Δt)'] = df['temperature_cleaned'].diff() / df['time_diff_seconds']

# 8. Recalculate the 'Rolling Variance (σ²ₚₕ)' for 'pH'
window_size = 5
df['Rolling Variance (σ²ₚₕ)'] = df['pH_cleaned'].rolling(window=window_size).var()

# 9. Recalculate the 'Short-Term Gradient (ΔNTU)' for 'Turbidity (NTU)'
df['Short-Term Gradient (ΔNTU)'] = df['turbidity_cleaned'].diff()

# 10. Recalculate the 'Moving Average Deviation' using 'tds_cleaned'
df['tds_rolling_mean'] = df['tds_cleaned'].rolling(window=window_size).mean()
df['Moving Average Deviation'] = df['tds_cleaned'] - df['tds_rolling_mean']

# 11. Display the head of the DataFrame to show all original, cleaned, and re-calculated feature-engineered columns
display(df[['time', 'Temperature (°C)', 'temperature_cleaned', 'Time Dissolve Solid (ppm)', 'tds_cleaned', 'pH', 'pH_cleaned', 'Turbidity (NTU)', 'turbidity_cleaned',
            'time_diff_seconds', 'Rate of Change (ΔT/Δt)', 'Rolling Variance (σ²ₚₕ)',
            'Short-Term Gradient (ΔNTU)', 'Moving Average Deviation']].head(10))

print("Dataset reloaded, cleaned, and all features re-calculated successfully.")

,time,Temperature (°C),temperature_cleaned,Time Dissolve Solid (ppm),tds_cleaned,pH,pH_cleaned,Turbidity (NTU),turbidity_cleaned,time_diff_seconds,Rate of Change (ΔT/Δt),Rolling Variance (σ²ₚₕ),Short-Term Gradient (ΔNTU),Moving Average Deviation
0,2025-12-03 22:45:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,NaN,NaN,NaN,NaN,NaN
1,2025-12-03 23:00:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,NaN,0.0,NaN
2,2025-12-03 23:15:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,NaN,0.0,NaN
3,2025-12-03 23:30:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,NaN,0.0,NaN
4,2025-12-03 23:45:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,0.0,0.0,0.0
5,2025-12-04 00:00:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,0.0,0.0,0.0
6,2025-12-04 00:15:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,0.0,0.0,0.0
7,2025-12-04 00:30:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,0.0,0.0,0.0
8,2025-12-04 00:45:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,0.0,0.0,0.0
9,2025-12-04 01:00:00+08:00,28.0,28.0,252.1596,252.1596,7.535385,7.535385,4,4.0,900.0,0.0,0.0,0.0,0.0


Dataset reloaded, cleaned, and all features re-calculated successfully.


In [ ]:
import pandas as pd

# Define the feature-engineered columns
#temperature-rate of change
#tds-moving average
#turbidity-short term gradient
#ph-rolling variance

feature_engineered_cols = [
    'Rate of Change (ΔT/Δt)',
    'Moving Average Deviation',
    'Short-Term Gradient (ΔNTU)',
    'Rolling Variance (σ²ₚₕ)',

]

# Define the original sensor columns (excluding 'time' which will be included separately)

sensor_cols = ['Temperature (°C)', 'Time Dissolve Solid (ppm)', 'pH', 'Turbidity (NTU)']

original_sensor_cols = [

    "temperature_cleaned",
    "tds_cleaned",
    "turbidity_cleaned",
    "pH_cleaned",

]

# --- File 1: time and featured engineered variables ---

# Select 'time' and all feature-engineered columns
df_file1 = df[['time'] + sensor_cols + original_sensor_cols + feature_engineered_cols + ['temp_fault', 'tds_fault', 'turbidity_fault', 'ph_fault',"fault detection"]].copy()

# Export to CSV
df_file1.to_csv('original.csv', index=False)
print("Exported 'original.csv' with time and engineered features.")

# --- File 2: time, original sensor data, and feature engineered variables ---

# Select 'time', original sensor columns, and feature-engineered columns
df_file2 = df[['time'] + original_sensor_cols + feature_engineered_cols + ['temp_fault', 'tds_fault', 'turbidity_fault', 'ph_fault',"fault detection"]].copy()

# Export to CSV
df_file2.to_csv('full_dataset_with_features.csv', index=False)
print("Exported 'full_dataset_with_features.csv' with original data and engineered features.")

Exported 'original.csv' with time and engineered features.
Exported 'full_dataset_with_features.csv' with original data and engineered features.


# Task
Modify the code in the first cell to create 'pH_cleaned' and 'turbidity_cleaned' columns by copying the original columns, replacing 0 with NaN, and then forward-filling them. After creating these columns, update the 'Rolling Variance (σ²ₚₕ)' calculation to use 'pH_cleaned' and the 'Short-Term Gradient (ΔNTU)' calculation to use 'turbidity_cleaned'. Finally, include 'pH_cleaned' and 'turbidity_cleaned' in the display statement.

## Modify code to create pH_cleaned and turbidity_cleaned columns

### Subtask:
Add code to create `pH_cleaned` and `turbidity_cleaned` columns by copying the original columns, replacing invalid values (0) with NaN, and then forward-filling them.


**Reasoning**:
The subtask requires updating the 'Rolling Variance (σ²ₚₕ)' calculation to use 'pH_cleaned' and the 'Short-Term Gradient (ΔNTU)' calculation to use 'turbidity_cleaned'.



**Reasoning**:
The subtask requires creating two new cleaned columns, `pH_cleaned` and `turbidity_cleaned`, by copying their respective original columns, replacing '0' values with NaN, and then forward-filling the NaN values.



## Summary:

### Data Analysis Key Findings
*   **Creation of Cleaned Columns**: The `pH_cleaned` and `turbidity_cleaned` columns were successfully generated by copying their original counterparts, replacing invalid '0' values with NaN, and then applying a forward-fill method to impute these missing values.
*   **Updated Feature Engineering Calculations**: The 'Rolling Variance ($\sigma^2_{pH}$)' calculation was updated to use the `pH_cleaned` column, and the 'Short-Term Gradient ($\Delta$NTU)' calculation now utilizes the `turbidity_cleaned` column.
*   **Display of Cleaned Data**: The modified and newly created cleaned columns (`pH_cleaned`, `turbidity_cleaned`) were successfully included in the final display of the DataFrame's head, confirming the changes.

### Insights or Next Steps
*   The process demonstrates the critical importance of data cleaning, as using cleaned data (`pH_cleaned`, `turbidity_cleaned`) leads to more accurate and reliable feature engineering outcomes (e.g., 'Rolling Variance', 'Short-Term Gradient').
*   Future analysis can now proceed with higher confidence in the quality of the 'pH' and 'Turbidity (NTU)' related features, as the common issue of '0' values representing invalid readings has been addressed.
